# Hello, TBB!

##### Sections
- _Code_: [Hello, TBB!](#Hello,-TBB!)

This set of Jupyter Notebooks contains a series of exercises that provide a taste of the features of the oneTBB library. 
To learn more about oneTBB, you can visit [oneTBB on Github](https://github.com/oneapi-src/oneTBB) or explore the Apress book 
[Pro TBB: C++ Parallel Programming with Threading Building Blocks (TBB)](https://www.apress.com/gp/book/9781484243978).

This module will familiarize you with the basic structure and flow of the tutorials, and allow you do a practice
build and run of a simple oneTBB application.

## Learning Objectives

* To become familiar with the structure and general flow of the tutorials
* Run a first example that uses oneTBB to execute two tasks in parallel.

## General Flow of the Tutorials

Typically each exercise in the oneTBB modules provides three sections: (1) a problem statement with
a working serial implementation, (2) the skeleton of a parallel implementation that describes steps
you need to perform to complete the exercise and (3) a completed solution. To get the most out of each
exercise, you should not look at the completed solution unless you really get stuck.

The remainder of this module provides a simple example that will follows the flow described
above and results in a simple TBB parallel application.

## Hello, TBB!

Let's start by creating an application that invokes a function, `sleep_then_print` twice. One 
invocation is ``sleep_then_print(2, "Hello")`` and the other is ``sleep_then_print(1, ", TBB!")``.  
When executed sequentially, one after the other, the first function call will sleep for 2 seconds 
then print "Hello" and the second function call will sleep for 1 more second and then print ", TBB!".

Inspect the code below - there are no modifications necessary. This code is our serial base case. Run the first 
cell to create the file, then run the cell below it to compile and execute the code.

1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/hello-tbb.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>

void sleep_then_print(unsigned sec, const char *str) {
  std::this_thread::sleep_for(std::chrono::seconds(sec));
  std::cout << str << std::flush;
}

int main() {
  std::cout << "Running sequentially:\n";
  auto st0 = std::chrono::high_resolution_clock::now();
  sleep_then_print(2, "Hello");
  sleep_then_print(1, ", TBB!");
  auto st1 = std::chrono::high_resolution_clock::now();
    
  std::cout << "\nSerial time   = " << 1e-9 * (st1-st0).count() << " seconds\n";
}

### Build and Run
Select the cell below and click Run ▶ to compile and execute the above example:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_hello-tbb.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_hello-tbb.sh; else ./scripts/run_hello-tbb.sh; fi

## Use tbb::parallel_invoke to add concurrency

Now we will add some concurrency to the code.

In the code cell below, the features of the TBB library are made available by including `tbb.h`. All oneTBB classes 
and functions are in namespace `tbb`.

An almost complete skeleton for the code is provide in the code cell. In the `main` function of our first example, 
the same two function calls are made again but we will use `tbb::parallel_invoke` to execute them concurrently. 

The interface of parallel_invoke needed for this example is shown below:

```cpp
template<typename... Functions>
void parallel_invoke(Functions&&... fs);
```

This function executes two or more function objects in parallel. When executed concurrently, the two calls to 
``sleep_then_print`` will take approximately 2 seconds in wallclock time to complete. And, as a consequence 
of their concurrent execution, the output from the second call is likely to be printed first (it only slept 
for 1 second before printing) and, as a result, the output might look jumbled, ", TBB!Hello".

For this exercise, complete the following steps:

1. Inspect the code cell below and make the following modifications.
  1. Remove the `//` before all the lines, except the one with "STEP 1" in it.
2. When the modifications are complete, click run ▶ to save the code to a file.
3. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file.

In [ ]:
%%writefile lab/hello-tbb-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>
#include <tbb/tbb.h>

void sleep_then_print(unsigned sec, const char *str) {
  std::this_thread::sleep_for(std::chrono::seconds(sec));
  std::cout << str << std::flush;
}

int main() {
  std::cout << "\nRunning in parallel:\n";
  auto pt0 = std::chrono::high_resolution_clock::now();
  // STEP 1: remove the '//' at the start of the lines to use parallel_invoke
  //  tbb::parallel_invoke(
  //  []() { 
    sleep_then_print(2, "Hello"); 
  // },
  //  []() { 
    sleep_then_print(1, ", TBB!"); 
  // });
  auto pt1 = std::chrono::high_resolution_clock::now();
    
  std::cout << "Parallel time = " << 1e-9 * (pt1-pt0).count() << " seconds\n";
}

### Build and Run
Select the cell below and click Run ▶ to compile and execute the above example:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_hello-tbb-parallel.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_hello-tbb-parallel.sh; else ./scripts/run_hello-tbb-parallel.sh; fi

## Solution (only peak if you need to)

In general, a fully complete solution will be provided, but will be hidden in the notebook.  You can expand the cells to see,
save, build and run the solution if needed.

In [ ]:
%%writefile solutions/hello-tbb-solved.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <chrono>
#include <iostream>
#include <thread>
#include <tbb/tbb.h>

void sleep_then_print(unsigned sec, const char *str) {
  std::this_thread::sleep_for(std::chrono::seconds(sec));
  std::cout << str << std::flush;
}

int main() {
  std::cout << "\nRunning in parallel:\n";
  auto pt0 = std::chrono::high_resolution_clock::now();
  // STEP 1: remove the '//' at the start of the lines to use parallel_invoke
  tbb::parallel_invoke(
    []() { 
      sleep_then_print(2, "Hello"); 
    },
    []() { 
      sleep_then_print(1, ", TBB!"); 
    });
  auto pt1 = std::chrono::high_resolution_clock::now();
    
  std::cout << "Parallel time = " << 1e-9 * (pt1-pt0).count() << " seconds\n";
}

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_hello-tbb-solved.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_hello-tbb-solved.sh; else ./scripts/run_hello-tbb-solved.sh; fi

## Next steps

You have now completed a first exercise! The remaining modules have similar structure but include exercises
that demand more than just uncommenting lines.

If you are ready, go to [the next module](../02_oneTBB_algorithms/oneTBB_algorithms.ipynb).